In [ ]:
import asyncio
from typing import Sequence

from autogen_agentchat.agents import BaseChatAgent, AssistantAgent
from autogen_agentchat.base import Response
from autogen_agentchat.messages import TextMessage, BaseChatMessage, BaseAgentEvent, HandoffMessage
from autogen_agentchat.teams import Swarm
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core import CancellationToken

In [ ]:
class PlannerAgent(BaseChatAgent):
    def __init__(self):
        super().__init__(name="planner", description="Creates writing outline.")

    @property
    def produced_message_types(self) -> Sequence[type[BaseChatMessage]]:
        return (TextMessage,)

    async def on_messages(self, messages, cancellation_token: CancellationToken) -> Response:
        topic = messages[-1].content if messages else "unspecified topic"
        outline = (
            f"Here's an outline for a blog post on **{topic}**:\n\n"
            "- Introduction\n"
            "- Background\n"
            "- Key Insights\n"
            "- Challenges\n"
            "- Conclusion\n"
            "\nI'll hand this off to the writer."
        )
        return Response(
            chat_message=TextMessage(content=outline, source=self.name),
            inner_messages=[
                HandoffMessage(content="Ready to write based on this outline.", source=self.name, target="writer")
            ]
        )

    async def on_reset(self, cancellation_token: CancellationToken) -> None:
        pass


In [ ]:
class WriterAgent(BaseChatAgent):
    def __init__(self):
        super().__init__(name="writer", description="Writes content from outline.")

    @property
    def produced_message_types(self) -> Sequence[type[BaseChatMessage]]:
        return (TextMessage,)

    async def on_messages(self, messages, cancellation_token: CancellationToken) -> Response:
        outline = messages[-1].content
        body = (
            "Here's a draft based on the outline:\n\n"
            "**Introduction:** AI is transforming the education sector...\n\n"
            "**Background:** Traditionally, learning was limited to...\n\n"
            "**Key Insights:** Personalized learning paths and instant feedback...\n\n"
            "**Challenges:** Equity, bias, and teacher roles must be considered...\n\n"
            "**Conclusion:** With careful adoption, AI can enhance learning at scale.\n"
        )
        return Response(chat_message=TextMessage(content=body, source=self.name))

    async def on_reset(self, cancellation_token: CancellationToken) -> None:
        pass


In [ ]:
async def run_writing_team():
    model_client = OpenAIChatCompletionClient(model="gpt-4o")

    # You can use AssistantAgent here instead of custom ones if needed
    planner = PlannerAgent()
    writer = WriterAgent()

    # Terminate after planner hands off to writer and writer responds
    termination = TextMentionTermination("Conclusion") | HandoffTermination("writer")

    team = Swarm(participants=[planner, writer], termination_condition=termination)

    # Run the team on a writing topic
    task = "The impact of AI on education"
    await Console(team.run_stream(task=task))

    await model_client.close()

await run_writing_team()
